In [ ]:
# Imports

import pandas as pd
import os
import csv
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from google_images_downloader import GoogleImagesDownloader
from sklearn.neighbors import NearestNeighbors
import IPython.display as display

# Sets iniciais de Dataset
O que eu preciso?
1) Um dataset que contém o id do usuário e um rating para um veículo, também com o ID dele
2) Um dataset que contém o id do veículo com suas características

In [ ]:
# Funções auxiliares

def ler_csv_para_dataframe(caminho_arquivo_csv):
    with open(caminho_arquivo_csv, 'r', newline='', encoding='utf-8') as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv)
        linhas = list(leitor_csv)        
    df = pd.DataFrame(linhas[1:], columns=linhas[0])
    return df
    
def renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova):
    for nome_arquivo in os.listdir(pasta):
        caminho_arquivo_antigo = os.path.join(pasta, nome_arquivo)
        if palavra_antiga in nome_arquivo:
            novo_nome_arquivo = nome_arquivo.replace(palavra_antiga, palavra_nova)
            caminho_arquivo_novo = os.path.join(pasta, novo_nome_arquivo)
            os.rename(caminho_arquivo_antigo, caminho_arquivo_novo)
    print("Arquivos renomeados com sucesso!")

def obter_nomes_apos_substring(pasta):
    nomes_extraidos = []
    for nome_arquivo in os.listdir(pasta):
        if "Scraped_Car_Review_" in nome_arquivo:
            parte_nome = nome_arquivo.split("Scraped_Car_Review_")[1].replace(".csv", "")
            nomes_extraidos.append(parte_nome)
    return nomes_extraidos

In [ ]:
# Tratamento dos arquivos nas pastas

pasta = './Data/'
palavra_antiga = "Reviews"
palavra_nova = "Review"
renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova)

palavra_antiga = "Scrapped"
palavra_nova = "Scraped"
renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova)

In [ ]:
# Upload dos datasets pegando quais carros tem em comum entre o dataset de consumidores e de veículos

marcas_de_carro = obter_nomes_apos_substring("./Data/")
todos_df = []

for marca in marcas_de_carro:
    file_path = f'./Data/Scraped_Car_Review_{marca}.csv'
    df = ler_csv_para_dataframe(file_path)
    df = df.drop(["Review_Title", "Review", "Review_Date", ""], axis=1)
    todos_df.append(df)

ratings = pd.concat(todos_df, ignore_index=True)
ratings = ratings.dropna()

# Tratativas

In [ ]:
# 1) Retirar o Author_Name e substituir por um ID unico para cada comprador

author_to_id = {}
for i, author in enumerate(ratings["Author_Name"].unique()):
    author_to_id[author] = i
ratings["userID"] = ratings["Author_Name"].apply(lambda author: author_to_id[author])

In [ ]:
# 2) Algumas tratativas do dataset

ratings['Company'] = ratings['Vehicle_Title'].str.split().str[1]
ratings['Vehicle_Title'] = ratings['Vehicle_Title'].str.split().str[2:]
ratings['Vehicle_Title'] = ratings['Vehicle_Title'].str.join(' ')
ratings['First_Word'] = ratings['Vehicle_Title'].str.split().str[0]
ratings = ratings.rename(columns={'First_Word': 'Model'})
ratings = ratings.drop("Vehicle_Title", axis=1)

ratings['carID'] = ratings['Company'].astype(str) + '-' + ratings['Model'].astype(str)
number_of_unique_ids = ratings['carID'].nunique()

cars = pd.read_csv('cars.csv')
cars['carID'] = cars['Company'].astype(str) + '-' + cars['Model'].astype(str)

# ratings.to_csv('ratings.csv')
# cars.to_csv('df_cars.csv')

# Algumas análises estatísticas

In [ ]:
n_ratings = len(ratings)
n_cars = len(ratings['carID'].unique())
n_users = len(ratings['userID'].unique())

print(f"Numero de ratings: {n_ratings}")
print(f"Numero de veiculos unicos: {n_cars}")
print(f"Numero de avaliadores unicos: {n_users}")
print(f"Rating medio por avaliador: {round(n_ratings/n_users, 2)}")
print(f"Numero medio de rating por veiculo: {round(n_ratings/n_cars, 2)}")

In [ ]:
ratings['Rating'] = ratings['Rating'].astype(float)
mean_rating = ratings.groupby('carID')[['Rating']].mean()
lowest_rated = mean_rating['Rating'].idxmin()
cars.loc[cars['carID'] == lowest_rated]
highest_rated = mean_rating['Rating'].idxmax()
cars.loc[cars['carID'] == highest_rated]
ratings[ratings['carID']==highest_rated]
ratings[ratings['carID']==lowest_rated]

car_stats = ratings.groupby('carID')[['Rating']].agg(['count', 'mean'])
car_stats.columns = car_stats.columns.droplevel()

ratings.head()
# cars.head()


In [ ]:
def create_matrix(df):
	
	N = len(df['userID'].unique())
	M = len(df['carID'].unique())
	
	user_mapper = dict(zip(np.unique(df["userID"]), list(range(N))))
	car_mapper = dict(zip(np.unique(df["carID"]), list(range(M))))
	
	user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userID"])))
	car_inv_mapper = dict(zip(list(range(M)), np.unique(df["carID"])))
	
	user_index = [user_mapper[i] for i in df['userID']]
	car_index = [car_mapper[i] for i in df['carID']]

	X = csr_matrix((df["Rating"], (car_index, user_index)), shape=(M, N))
	
	return X, user_mapper, car_mapper, user_inv_mapper, car_inv_mapper
	
X, user_mapper, car_mapper, user_inv_mapper, car_inv_mapper = create_matrix(ratings)


In [ ]:
def find_similar_movies(car_id, X, k, metric='euclidean', show_distance=False):
	
	neighbour_ids = []
	
	car_ind = car_mapper[car_id]
	car_vec = X[car_ind]
	k+=1
	kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
	kNN.fit(X)
	car_vec = car_vec.reshape(1,-1)
	neighbour = kNN.kneighbors(car_vec, return_distance=show_distance)
	for i in range(0,k):
		n = neighbour.item(i)
		neighbour_ids.append(car_inv_mapper[n])
	neighbour_ids.pop(0)
	return neighbour_ids


car_titles = dict(zip(cars['carID'], cars['carID']))

car_id = 'Mercury-Cougar'

similar_ids = find_similar_movies(car_id, X, k=3)
car_title = car_titles[car_id]

for i in similar_ids:
	downloader = GoogleImagesDownloader(browser="chrome", show=False, debug=False, quiet=False, disable_safeui=False) 
	downloader.download(car_titles[i], limit=1, destination="./carros-sugeridos")
	downloader.close()


In [ ]:
# print(f"Como voce gostou de {car_title}, voce tambem pode gostar dos:")
image_path0 = f'./carros-sugeridos/{car_titles[similar_ids[0]]}/{car_titles[similar_ids[0]]}_0.jpg'
image_path1 = f'./carros-sugeridos/{car_titles[similar_ids[1]]}/{car_titles[similar_ids[1]]}_0.jpg'
image_path2 = f'./carros-sugeridos/{car_titles[similar_ids[2]]}/{car_titles[similar_ids[2]]}_0.jpg'

# md = f'<img src = "{image_path0}" width="300px">'
x = 10

expr = f"'O valor de x é {x}'"
print(eval(expr))